# Secret sharing reference communication benchmark
In this notebook we try to count the amount of communication that is needed between workers and the executor to get an idea for how much communication is a bottleneck. In this reference example, every computational step is coordinated by the notebook which acts as a virtual central sever, in the real world this is different. 

## Conclusion
This is an artifical example in that all communication goes through 'me' and there is no communication between the workers. We counted **26 messages** were received. But this number does not reflect the real case in 2 ways:
* No message is sent/received when data moves from the worker to the central orchestrator, this will happen in the real case.
* All communication is with the central orchestrator and not worker-to-worker.

We got the feeling that the virtualworkers mimic the API 1-to-1, but the communication doesn't actually match with real workers. See also: https://github.com/OpenMined/PySyft/issues/5218

So: to get a better picture we would need to investigate this in a more realistic setup with actual workers instead of virtual ones running in the same python instance.

In [1]:
import syft as sy
import torch as th

hook = sy.TorchHook(th)

In [2]:
class LoggingWorker(sy.VirtualWorker):
    """
    A virtual worker that logs sent and received messages.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.send_history = list()
        self.recv_history = list()
        
    def send_msg(self, message, location: "BaseWorker") -> object:
        """Implements the logic to send messages.
        The message is serialized and sent to the specified location. The
        response from the location (remote worker) is deserialized and
        returned back.
        Every message uses this method.
        Args:
            msg_type: A integer representing the message type.
            message: A Message object
            location: A BaseWorker instance that lets you provide the
                destination to send the message.
        Returns:
            The deserialized form of message from the worker at specified
            location.
        """
        if self.verbose:
            print(f"worker {self} sending {message} to {location}")
            
        # Step 1: serialize the message to a binary
        bin_message = sy.serde.serialize(message, worker=self)

        # Step 2: send the message and wait for a response
        bin_response = self._send_msg(bin_message, location)

        # Step 3: deserialize the response
        response = sy.serde.deserialize(bin_response, worker=self)
        
        self.send_history.append((message, len(bin_message)))
        
        return response
    
    def recv_msg(self, bin_message: bin) -> bin:
        """Implements the logic to receive messages.
        The binary message is deserialized and routed to the appropriate
        function. And, the response serialized the returned back.
        Every message uses this method.
        Args:
            bin_message: A binary serialized message.
        Returns:
            A binary message response.
        """
        # Step 0: deserialize message
        msg = sy.serde.deserialize(bin_message, worker=self)

        # Step 1: save message and/or log it out
        if self.log_msgs:
            self.msg_history.append(msg)

        if self.verbose:
            print(
                f"worker {self} received {type(msg).__name__} {msg.contents}"
                if hasattr(msg, "contents")
                else f"worker {self} received {type(msg).__name__}"
            )
        self.recv_history.append((msg, len(bin_message)))

        # Step 2: route message to appropriate function

        response = None
        for handler in self.message_handlers:
            if handler.supports(msg):
                response = handler.handle(msg)
                break
        # TODO(karlhigley): Raise an exception if no handler is found

        # Step 3: Serialize the message to simple python objects
        bin_response = sy.serde.serialize(response, worker=self)

        return bin_response


# Step one: defining the parties

Alice and bob will be the data owners.
There is an implicit third party called "me" which is the party which runs the code, holds the pointers to the data, and will also be the crypto provider because we haven't defined a separate crypto provider.

In [3]:
alice = LoggingWorker(hook, id='alice')
bob = LoggingWorker(hook, id='bob')

alice.add_workers([bob])
bob.add_workers([alice])

Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<LoggingWorker id:bob #objects:0>

# Step two: storing the data
Two tensors _x_ and _y_ are defined and sent to Alice and Bob. 

In [4]:
x = th.Tensor([1,2,3])
y = th.Tensor([4,5,6])

x_p = x.send(alice)
y_p = y.send(bob)

alice._objects

{30156683280: tensor([1., 2., 3.])}

# Step three: sharing secrets
`x` and `y` are converted to fixed precision values and encrypted into shared secrets.
Alice and Bob now contain an AdditiveSharingTensor, which contains pointers to all the secret shares at the different locations.

In [5]:
x_share = x_p.fix_prec().share(bob, alice)
y_share = y_p.fix_prec().share(bob, alice)

alice._objects

{30156683280: tensor([1., 2., 3.]),
 49108460429: tensor([-7861231453757190850,   162443217343479452,  1234102003147360136]),
 92109403716: (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:61923783235 -> bob:55329268038]
 	-> [PointerTensor | me:62375302071 -> alice:49108460429]
 	*crypto provider: me*,
 71359208984: tensor([ 6255037113097608685,   -88234053770322261, -5700560230518340298])}

# Step four: collecting the pointers for the shared secrets and performing the analysis
In order to perform computations on the AdditiveSharingTensors they need to be moved to the same party by calling `get`. Mind you, this does not move the secret values, merely the pointers to the cryptographical pieces that are located at the different parties.

After moving the tensors we can perform the analysis. The result of the analysis will be another AdditiveSharingTensor.

In [6]:
secret_result = (x_share.get() + y_share.get())/2
secret_result

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:97430375456 -> bob:17900866039]
	-> [PointerTensor | me:34459115429 -> alice:22120326466]
	*crypto provider: me*

# Step five: Getting the result of the computation
After performing the computation it is time to decrypt the result and convert it back into a float.

In [7]:
result = secret_result.get().float_prec()
result

tensor([2.5000, 3.5000, 4.5000])

## Communication counting
We count the number of received messages, as in this virtual example this is the only communication there seems to be. Everything seems to go through 'me', the workers do not send anything to each other.

In [8]:
num_messages = len(bob.recv_history) + len(alice.recv_history)
print(f'Communication happened {num_messages} times')
average_size = (sum(size for msg, size in bob.recv_history) 
                + sum(size for msg, size in alice.recv_history)) / num_messages
print(f'Average message size: {average_size} bytes')

Communication happened 26 times
Average message size: 150.5 bytes


In [9]:
alice.recv_history

[((ObjectMessage tensor([1., 2., 3.])), 385),
 ((TensorCommandMessage ComputationAction[3776333929 = tensor([1., 2., 3.]).fix_prec(owner=<VirtualWorker id:me #objects:0>)]),
  116),
 ((TensorCommandMessage ComputationAction[92109403716 = (Wrapper)>FixedPrecisionTensor>tensor([1000, 2000, 3000]).share(<LoggingWorker id:bob #objects:1>, <LoggingWorker id:alice #objects:1>, protocol=snn, field=None, dtype=None, crypto_provider=None, requires_grad=False)]),
  187),
 ((ObjectMessage tensor([-7861231453757190850,   162443217343479452,  1234102003147360136])),
  398),
 ((ForceObjectDeleteMessage [3776333929]), 34),
 ((ObjectMessage tensor([ 6255037113097608685,   -88234053770322261, -5700560230518340298])),
  398),
 ((ObjectRequestMessage (92109403716, None, '')), 42),
 ((TensorCommandMessage ComputationAction[65375042530 = tensor([-7861231453757190850,   162443217343479452,  1234102003147360136]).__add__(tensor([ 6255037113097608685,   -88234053770322261, -5700560230518340298]))]),
  133),
 

Alice' send history is empty, this is strange because how did we get the result with her sending the result back to 'me'? And how did Alice secretly share her tensor with Bob? 

In [10]:
alice.send_history

[]

Only if we move a tensor from alice to bob we see this in the logs, so inter-worker communication is logged, but the communication from a worker to 'me' is **not** logged.

In [11]:
result_p = result.send(alice)
result_p.move(bob)

(Wrapper)>[PointerTensor | me:68732150258 -> bob:90086845621]

In [12]:
alice.send_history

[((ObjectMessage tensor([2.5000, 3.5000, 4.5000])), 385)]